In [ ]:
%matplotlib inline
import sys
sys.path.append("..")
from pathlib import Path
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from skimage import io
from sklearn.manifold import TSNE

import pythd

In [ ]:
DATA_DIR = Path("..") / Path("data")

POST_IMAGE_NAME = "hurricane-florence_00000419_post_disaster.png"
POST_IMAGE_PATH = DATA_DIR / POST_IMAGE_NAME
PRE_IMAGE_PATH = DATA_DIR / POST_IMAGE_NAME.replace("post", "pre")
POST_LABEL_PATH = DATA_DIR / POST_IMAGE_NAME.replace("png", "json")

In [ ]:
def plot_images(images, dpi=300.0, **kwargs):
    """Show multiple images in one plot"""
    fig, ax = plt.subplots(1, len(images), dpi=dpi)
    for i, img in enumerate(images):
        ax[i].imshow(img, **kwargs)
        ax[i].axis("off")
    return (fig, ax)

def chip_image(img, shape, stride=None):
    shape = (shape, shape) if isinstance(shape, int) else shape
    stride = shape if (stride is None) else stride
    stride = (stride, stride) if isinstance(stride, int) else stride
    cw, ch = shape
    sx, sy = stride
    
    h, w, c = (img.shape[0], img.shape[1], img.shape[2])
    ncol = (cw * ch * c) + 2
    
    maxx = w - cw
    maxy = h - ch
    nx = 1 + (maxx // sx)
    ny = 1 + (maxy // sy)
    nrow = nx * ny
    chips = np.zeros((nrow, ncol))
    row = 0
    columns = ["x", "y"]
    columns += ["pixel {}".format(i+1) for i in range(ch*cw*c)]
    
    for y in range(0, h-sy, sy):
        for x in range(0, w-sx, sx):
            chips[row, 2:] = img[y:(y+ch), x:(x+cw), :].flatten()
            chips[row, 0] = x
            chips[row, 1] = y
            row += 1
            
    return pd.DataFrame(chips, columns=columns)

In [ ]:
pre_img = io.imread(str(PRE_IMAGE_PATH))
post_img = io.imread(str(POST_IMAGE_PATH))

In [ ]:
pre_chipped = chip_image(pre_img, 16,16)
print(pre_chipped.shape)

In [ ]:
_ = plot_images((pre_img, post_img))

In [ ]:
filt = pythd.filter.ScikitLearnFilter(TSNE, n_components=2)
f_x = filt(pre_chipped.iloc[:, 2:])